# Extracción de características HOG
Este notebook aplica el descriptor HOG (Histogram of Oriented Gradients) a las imágenes normalizadas

In [3]:
import os
import glob
import numpy as np
import cv2
from skimage.feature import hog
from tqdm import tqdm
import matplotlib.pyplot as plt

# Rutas base
BASE_DIR = os.path.dirname(os.path.abspath('.'))
NORMALIZADAS_DIR = os.path.join(BASE_DIR, 'normalizacion', 'normalizadas')
HOG_DIR = os.path.join(BASE_DIR, 'HOG', 'hog_npy')

# Parámetros HOG
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)

print(f"Ruta de imágenes normalizadas: {NORMALIZADAS_DIR}")
print(f"Ruta de salida HOG: {HOG_DIR}")

Ruta de imágenes normalizadas: /workspaces/fruit-detector-pricer/ProyectoFinal/normalizacion/normalizadas
Ruta de salida HOG: /workspaces/fruit-detector-pricer/ProyectoFinal/HOG/hog_npy


In [4]:
def procesar_imagenes(origen_split):
    split_path = os.path.join(NORMALIZADAS_DIR, origen_split)
    if not os.path.exists(split_path):
        print(f"❌ Carpeta no encontrada: {split_path}")
        return
    
    # Crear directorio de salida si no existe
    out_split_path = os.path.join(HOG_DIR, origen_split)
    os.makedirs(out_split_path, exist_ok=True)
    
    # Obtener lista de clases (carpetas de frutas)
    clases = [d for d in os.listdir(split_path) 
             if os.path.isdir(os.path.join(split_path, d))]
    
    for clase in clases:
        # Crear directorio de salida para la clase
        clase_in_dir = os.path.join(split_path, clase)
        clase_out_dir = os.path.join(out_split_path, clase)
        os.makedirs(clase_out_dir, exist_ok=True)
        
        # Procesar cada imagen
        imagenes = glob.glob(os.path.join(clase_in_dir, '*_recorte.png'))
        
        for img_path in tqdm(imagenes, desc=f"Procesando {clase} en {origen_split}"):
            # Leer imagen
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                print(f"⚠️ No se pudo leer la imagen: {img_path}")
                continue
                
            # Calcular HOG
            features = hog(img, 
                          orientations=orientations,
                          pixels_per_cell=pixels_per_cell,
                          cells_per_block=cells_per_block,
                          visualize=False)
            
            # Guardar características HOG
            nombre_base = os.path.basename(img_path).replace('_recorte.png', '')
            out_path = os.path.join(clase_out_dir, f"{nombre_base}_hog.npy")
            np.save(out_path, features)

# Procesar imágenes de train y test
for split in ['train', 'test']:
    procesar_imagenes(split)

print("\n✓ Procesamiento HOG completado")
print("Las imágenes HOG han sido guardadas en la carpeta hog_features/")

Procesando Pineapple Mini 1 en test: 100%|██████████| 148/148 [00:00<00:00, 158.81it/s]


✓ Procesamiento HOG completado
Las imágenes HOG han sido guardadas en la carpeta hog_features/
